# Playing with Contrast Enhancement, Histogram Equalization
stough 202-

We've now seen how to improve image contrast through the application of transfer functions like [windowing, power/gamma, log, and others](./enhance_histeq.ipynb). We've also seen how [Histogram Equalization](./enhance_histeq.ipynb) creates an ideal transfer function in some sense--that of leading to a uniform histogram. Use these other demos in order to answer the following questions

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, IntSlider
from scipy.interpolate import interp1d

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)

from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_lab_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

## Find an Image

Find **at least** two reasonably inoffensive images (maybe a little offensive) to test on. One key here is that because we're doing contrast enhancement, you might err on the side of lower-contrast images. You don't have to do every test on both, but it wouldn't be difficult.

In [2]:
I = plt.imread('../dip_pics/build.jpg')
vis_hists(I)
arr_info(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((640, 960, 3), dtype('uint8'), 0, 255)

## 1. Parametric Transfer Functions 

- Use `vis_hists` to **show the histogram of an image**. 
- With that visual assistance consider what parametric transfer function (power/gamma, log, exponent) might be helpful in this image's case.
- **Justify your choice** with written description
- **Display the result** of applying the transfer function of choice to the image. 
- You can use `vis_pair` to show both the original and the presumeably contrast-enhanced images together.
- **Explain how**, with written description, the figures you generate either do or do not reflect your justification from before. You can even **show a zoomed-in section** that illustrates your contentions.

In [3]:
# Use vis_hists to show the histogram of an image
vis_hists(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This photo have a high concentration of pixels on the range between [200, 255]. Therefore, to add more contrast, we need to mapping pixels in that range to a broader range. To accomplish this, we could choose concave up, increasing functions to apply on this image. So it could be power/gamma function with $\gamma > 1$, or exponent function.

In [4]:
# Apply function to the image 
def power_img(I, gamma = 1):
    factor = 255.0 / np.power(255.0, gamma)
    J = factor * np.power(I.astype('double'), gamma)
    return J.astype('uint8')

J = power_img(I, 5)

In [5]:
vis_pair(I, J)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
vis_hists(J)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Alternative Colorspace

**Show** the same parametric contrast enhancement as in Part 1, but performed **only on Luminance/Y/Value** dimension after conversion into  [Lab](../Color/color_Lab.ipynb)/[YCbCr](../Color/color_YCbCr.ipynb)/[HSV](../Color/color_HSV.ipynb). **Explain your results**. You only need to pick one of the three alternative colorspaces mentioned. Remember, you'll need to: 

- Convert to the alternative colorspace using the appropriate [skimage.color](https://scikit-image.org/docs/dev/api/skimage.color.html) function.
- Change only the one channel you're working with. Be sure you're not producing out-of-bounds values for the particular colorspace you choose.
- Convert back using the appropriate inverse from `skimage.color`.

In [7]:
K = color.rgb2lab(I)
arr_info(K)

((640, 960, 3), dtype('float64'), -66.51691412480429, 100.0)

In [8]:
gamma = 5
fac = 100.0 / np.power(100.0, gamma)
K[...,0] = fac * np.power(K[...,0],gamma)

In [9]:
vis_pair(I, color.lab2rgb(K))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
vis_hists(color.lab2rgb(K))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 3. Histogram Equalization

Again you can use `vis_hists`, `vis_pair`, `vis_[rgb,lab,hsv,ybr]_cube` to visualize results.

- **Histogram-equalize an image and show** the resulting change in the image (as in, "Before" and "After" with `vis_pair`).
- **Explain how**, with written description, the figures you generate do or do not reflect good contrast enhancement. Are there aritifacts in the corrected image that make it not an "improvement" in your eyes? 
- Try with two alternative colorspaces: **Perform the histogram equalization on only the Luminance/Y/Value** dimension after conversion into  [Lab](../Color/color_Lab.ipynb)/[YCbCr](../Color/color_YCbCr.ipynb)/[HSV](../Color/color_HSV.ipynb) space, and **show the results**. 
- **Explain your results**.

In [11]:
M = color.rgb2ycbcr(I)

bins=np.arange(16, 235)

freq, bins = np.histogram(M[...,0].ravel(), bins=bins)

freq = freq/freq.max()
cdf = np.cumsum(freq)
cdf = cdf/cdf.max()
func = interp1d(bins[:-1], cdf, fill_value='extrapolate')

M[...,0] = func(M[...,0])
#M[...,0] = (M[...,0]-M[...,0].min()) / (M[...,0].max()-M[...,0].min()) * 255
M[...,0] = M[...,0] / M[...,0].max() * 255

Mrgb = color.ycbcr2rgb(M)
Mrgb = Mrgb * 255 / (Mrgb.max() - Mrgb.min())
Mrgb = np.uint8(Mrgb + abs(Mrgb.min()))
print(arr_info(Mrgb))

vis_pair(I, Mrgb, second_title='Hist Equalized')

((640, 960, 3), dtype('uint8'), 0, 255)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
L = color.rgb2hsv(I)
bins=len(np.unique(L[...,2].ravel()))
freq, bins = np.histogram(L[...,2].ravel(), bins=bins)
# Check bins
freq = freq/freq.max()
cdf = np.cumsum(freq)
cdf = cdf/cdf.max()
func = interp1d(bins[:-1], cdf, fill_value='extrapolate') 
L[...,2] = func(L[...,2])
L[...,2] = L[...,2] / L[...,2].max()
vis_pair(I, color.hsv2rgb(L), second_title='Hist Equalized')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …